In [38]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import cv2

In [39]:
# Load the Excel dataset
excel_file = 'labels.csv'  # Path to your Excel file
df = pd.read_csv(excel_file)

# Preview dataset
print(df.head())

                Filename  Label       Species
0  20160928-140314-0.jpg      0  Chinee apple
1  20160928-140337-0.jpg      0  Chinee apple
2  20160928-140731-0.jpg      0  Chinee apple
3  20160928-140747-0.jpg      0  Chinee apple
4  20160928-141107-0.jpg      0  Chinee apple


In [40]:
# Set the image folder path and target image size
image_folder = 'images/'  # Folder containing images
image_size = (128, 128)  # Resize all images to 128x128

In [41]:
# Function to load and preprocess images
def load_and_preprocess_images(image_folder, filenames, image_size):
    images = []
    for filename in filenames:
        image_path = os.path.join(image_folder, filename)
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.resize(image, image_size)  # Resize image
            images.append(image)
        else:
            print(f"Image not found: {filename}")
    return np.array(images)

In [42]:
# Load images using filenames from Excel file
images = load_and_preprocess_images(image_folder, df['Filename'], image_size)

In [43]:
# Encode the labels (weed varieties)
le = LabelEncoder()
labels = le.fit_transform(df['Species'])
labels = to_categorical(labels)  # Convert to one-hot encoding

# Split dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [44]:
# Normalize pixel values to [0, 1] range
X_train = X_train / 255.0
X_test = X_test / 255.0

# CNN model definition
def create_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(le.classes_), activation='softmax')  # Output layer
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [45]:
# Create the CNN model
input_shape = (image_size[0], image_size[1], 3)  # 3 channels for RGB
model = create_model(input_shape)

# Train the model
batch_size = 32
epochs = 10
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

c:\Users\Kishan raj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 88s 370ms/step - accuracy: 0.1883 - loss: 2.0063 - val_accuracy: 0.2868 - val_loss: 1.9746
Epoch 2/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 67s 354ms/step - accuracy: 0.3559 - loss: 1.6867 - val_accuracy: 0.4398 - val_loss: 1.4278
Epoch 3/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 65s 342ms/step - accuracy: 0.4867 - loss: 1.3566 - val_accuracy: 0.3373 - val_loss: 1.9039
Epoch 4/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 48s 251ms/step - accuracy: 0.4842 - loss: 1.3953 - val_accuracy: 0.4443 - val_loss: 1.4220
Epoch 5/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 45s 237ms/step - accuracy: 0.5962 - loss: 1.1020 - val_accuracy: 0.5141 - val_loss: 1.2300
Epoch 6/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 50s 261ms/step - accuracy: 0.6853 - loss: 0.8926 - val_accuracy: 0.4339 - val_loss: 1.6675
Epoch 7/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 51s 270ms/step - accuracy: 0.7658 - loss: 0.6720 - val_accuracy: 0.5067 - val_loss: 1.6344
Epoch 8/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 51s 266ms/step - accuracy: 0.7932 - loss: 0

In [46]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc * 100:.2f}%")

53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - accuracy: 0.4964 - loss: 2.4323
Test accuracy: 50.39%


In [47]:
# Save the model
model.save('weed_classification_model.h5')

In [48]:
# Predict on a test image (example)
sample_image = "images/20170711-115741-0.jpg"
sample_image = X_test[0].reshape(1, image_size[0], image_size[1], 3)
prediction = model.predict(sample_image)
predicted_class = le.inverse_transform([np.argmax(prediction)])
print(f"Predicted weed variety: {predicted_class[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
Predicted weed variety: Chinee apple


In [50]:
import json

# Save the class labels (weed species) to a JSON file
class_labels = list(le.classes_)  # List of species names
with open('class_labels.json', 'w') as f:
    json.dump(class_labels, f)